<a href="https://colab.research.google.com/github/nadavmoyal/Final_Project-FetalCns/blob/main/Fetal_Cns_Final_reg_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import models
from PIL import Image
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, Dataset

In [2]:
# Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

Mounted at /content/drive
Google Drive mounted successfully.


In [3]:

# Determine the available hardware (CPU or GPU) and set the PyTorch device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [4]:

# Read the list of images from the CSV file
train_csv = "/content/drive/MyDrive/FinalProject/our_training_set_pixel_size_and_HC_rand.csv"
train_csv_df = pd.read_csv(train_csv)
print("CSV file loaded:")
print(train_csv_df.head())


CSV file loaded:
      filename  pixel size(mm)  head circumference (mm)
0  444_2HC.png        0.131226                   174.30
1   417_HC.png        0.124412                   169.84
2   617_HC.png        0.145178                   189.78
3   121_HC.png        0.060706                    79.86
4   098_HC.png        0.078805                    78.50


In [5]:
# Compute min and max values of the head circumference
min_hc = train_csv_df['head circumference (mm)'].min()
max_hc = train_csv_df['head circumference (mm)'].max()
print(f"Min HC: {min_hc}, Max HC: {max_hc}")


Min HC: 56.81, Max HC: 346.4


In [6]:
# Normalize the labels in the dataset
train_csv_df['Normalized_HC'] = (train_csv_df['head circumference (mm)'] - min_hc) / (max_hc - min_hc)

In [7]:
# # Save the normalized dataset
output_csv = "/content/drive/MyDrive/FinalProject/our_test_set_pixel_size_and_HC_rand_no_index.csv"
train_csv_df.to_csv(output_csv, index=False)
print("Normalized CSV file saved.")
print(train_csv_df.head())

Normalized CSV file saved.
      filename  pixel size(mm)  head circumference (mm)  Normalized_HC
0  444_2HC.png        0.131226                   174.30       0.405712
1   417_HC.png        0.124412                   169.84       0.390310
2   617_HC.png        0.145178                   189.78       0.459166
3   121_HC.png        0.060706                    79.86       0.079595
4   098_HC.png        0.078805                    78.50       0.074899


In [8]:
# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        pixel_size = self.data_frame.iloc[idx, 1]
        hc = self.data_frame.iloc[idx, 3]  # Normalized head circumference
        sample = {'image': image, 'pixel_size': pixel_size, 'hc': hc}
        return sample

In [9]:
# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
# Load the dataset
root_dir = "/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/"
dataset = CustomDataset(csv_file=output_csv, root_dir=root_dir, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [11]:
class CNNRegression(nn.Module):
    def __init__(self):
        super(CNNRegression, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        cnn_out_features = self.cnn.fc.in_features  # Get the number of output features from the CNN
        self.cnn.fc = nn.Identity()  # Remove the last fully connected layer
        self.fc1 = nn.Linear(cnn_out_features + 1, 128)  # Adjust input features
        self.fc2 = nn.Linear(128, 1)

    def forward(self, image, pixel_size):
        x1 = self.cnn(image)
        x2 = pixel_size.view(-1, 1)  # Ensure x2 has the shape (batch_size, 1)
        x = torch.cat((x1, x2), dim=1)  # Concatenate along the feature dimension
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x


In [12]:
# Initialize the model, loss function, and optimizer
model = CNNRegression().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 80.8MB/s]


In [ ]:
# Lists to store the loss values
train_losses = []
val_losses = []

# Training loop
num_epochs = 16
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data in train_loader:
        images = data['image'].to(device)
        pixel_sizes = data['pixel_size'].float().to(device)
        targets = data['hc'].float().to(device)

        optimizer.zero_grad()
        outputs = model(images, pixel_sizes)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in val_loader:
            images = data['image'].to(device)
            pixel_sizes = data['pixel_size'].float().to(device)
            targets = data['hc'].float().to(device)

            outputs = model(images, pixel_sizes)
            loss = criterion(outputs, targets.unsqueeze(1))
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}")

# Plot the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.show()

Epoch 1/16, Training Loss: 0.04631346217938699
Epoch 1/16, Validation Loss: 0.008741979900514707


In [ ]:
# Evaluate model performance
y_true = []
y_pred = []
model.eval()
with torch.no_grad():
    for data in val_loader:
        images = data['image'].to(device)
        pixel_sizes = data['pixel_size'].float().to(device)
        targets = data['hc'].float().to(device)

        outputs = model(images, pixel_sizes)
        y_true.extend(targets.cpu().numpy())
        y_pred.extend(outputs.cpu().numpy())

mse = mean_squared_error(y_true, y_pred)
print(f"Mean Squared Error: {mse}")


In [ ]:
def predict_head_circumference(image_path, pixel_size, model, min_hc, max_hc):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    pixel_size_tensor = torch.tensor([pixel_size]).float().view(1, 1)  # Ensure it has the shape (1, 1)

    with torch.no_grad():
        output = model(image_tensor.to(device), pixel_size_tensor.to(device))

    predicted_normalized_hc = output.item()
    predicted_hc = predicted_normalized_hc * (max_hc - min_hc) + min_hc  # De-normalize the prediction
    return predicted_hc


In [ ]:
# Read the list of images from the CSV file
test_csv = "/content/drive/MyDrive/FinalProject/our_test_set_pixel_size_and_HC_rand.csv"
test_csv_df = pd.read_csv(test_csv)
print("CSV file loaded:")
print(test_csv_df.head())


In [ ]:
# Compute min and max values of the head circumference
min_hc = test_csv_df['head circumference (mm)'].min()
max_hc = test_csv_df['head circumference (mm)'].max()
print(f"Min HC: {min_hc}, Max HC: {max_hc}")

In [ ]:
# Normalize the labels in the dataset
test_csv_df['Normalized_HC'] = (test_csv_df['head circumference (mm)'] - min_hc) / (max_hc - min_hc)

In [ ]:
# # Save the normalized dataset
output_csv = "/content/drive/MyDrive/FinalProject/our_test_set_pixel_size_and_HC_rand_no_index.csv"
test_csv_df.to_csv(output_csv, index=False)
print("Normalized CSV file saved.")
print(test_csv_df.head())

In [ ]:
# Ensure the min_hc and max_hc are defined
min_hc = test_csv_df['head circumference (mm)'].min()
max_hc = test_csv_df['head circumference (mm)'].max()
num_images_to_test = len(test_csv_df)
indices = range(num_images_to_test)  # Generate a sequence of indices from 0 to num_images_to_test-1

# Initialize a list to store the results
results = []
absolute_errors = []
counter = 0
percentage_error_counter = 0  # Counter for errors less than 5%
threshold_param = 0.05

# Variables to track the maximum absolute error and corresponding details
max_absolute_error = -1
max_error_image = ""
max_error_predicted = -1
max_error_ground_truth = -1

# Loop through the selected images
for idx in indices:
    image_name = test_csv_df.iloc[idx, 0]
    image_path = os.path.join("/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/", image_name)
    ground_truth_circumference = test_csv_df.iloc[idx, 2]  # Assuming the third column contains the head circumference labels
    pixel_size = test_csv_df.iloc[idx, 1]  # Assuming the second column contains the pixel size

    # Call the function to predict head circumference from the image
    predicted_circumference = predict_head_circumference(image_path, pixel_size, model, min_hc, max_hc)

    # Calculate the absolute error
    absolute_error = abs(predicted_circumference - ground_truth_circumference)
    absolute_errors.append(absolute_error)

    # Calculate the 5% threshold for the ground truth circumference
    threshold = threshold_param * ground_truth_circumference

    # Check if the absolute error is less than the 5% threshold
    if absolute_error < threshold:
        percentage_error_counter += 1

    # Track the maximum absolute error
    if absolute_error > max_absolute_error:
        max_absolute_error = absolute_error
        max_error_image = image_path
        max_error_predicted = predicted_circumference
        max_error_ground_truth = ground_truth_circumference

    # Create the result string
    result_string = (f"Image: {image_path}\n"
                     f"Predicted Head Circumference: {predicted_circumference}\n"
                     f"Ground Truth Head Circumference: {ground_truth_circumference}\n"
                     f"Pixel Size: {pixel_size}\n"
                     f"Absolute Error: {absolute_error}\n"
                     f"5% Threshold: {threshold}\n"
                     "****************************\n")

    # Append the result to the list
    results.append([image_name, result_string, absolute_error])


# Convert the results to a DataFrame and save to a CSV file
results_df = pd.DataFrame(results, columns=['Image', 'Result', 'Absolute Error'])
results_csv_path = "/content/drive/MyDrive/FinalProject/results.csv"
results_df.to_csv(results_csv_path, index=False)

# Print summary
print(f"Results saved to {results_csv_path}")
print(f"Total Images with Error < {threshold_param * 100}% of Ground Truth: {percentage_error_counter}")
print(f"Percentage: {percentage_error_counter / num_images_to_test * 100:.2f}%")

# Print the image with the biggest absolute error
print("Image with the biggest absolute error:")
print(f"Image: {max_error_image}")
print(f"Predicted Head Circumference: {max_error_predicted}")
print(f"Ground Truth Head Circumference: {max_error_ground_truth}")
print(f"Absolute Error: {max_absolute_error}")
